In [1]:
# import statements
import os
import glob
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as col
from astropy.io import fits
from scipy import ndimage
from photutils import centroid_sources, centroid_com, centroid_1dg, centroid_2dg
from calibFuncs import *

from astropy.visualization import LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize

1. Gather Clio data
2. Outline of reduction script:
3. linearity correction
4. bad pixel correction
5. sky subtraction via nodding pairs
6. coarse registration, then fine registration via IDL

In [2]:
root = os.getcwd()
datadir = 'C:/Users/willi/Dropbox (Amherst College)/Research/Follette-Lab/Thesis-data/William/trapezium_data/clio_astro'

In [3]:
badpixelpath = datadir + '/../../Clio2_reduction_files'

In [4]:
# if old data intermediate data is around, delete 
for im in glob.glob(datadir+'\\*_LBP*.fit*'):
    os.remove(im)

In [5]:
runClioReduction(datadir, badpixelpath)

100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [00:27<00:00,  3.20it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|                                                                                           | 0/22 [00:00<?, ?it/s]

putting files in an existing folder.


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [15:07<00:00, 41.23s/it]

Done reducing the files


In [20]:
datadir = 'C:/Users/willi/Dropbox (Amherst College)/Research/Follette-Lab/Thesis-data/William/trapezium_data/clio_astro/reduced'
reduced_datalists, darks = sortData(datadir, filesdeep='')

In [ ]:
## TESTING ##

In [29]:
def clioCoadd(reduced_data, savepath):
    '''
    coadds reduced clio data
    '''
    if os.path.exists(savepath) is False:
        os.mkdir(savepath)
        print('made dir: '+savepath)
    for imlist in tqdm.tqdm(reduced_data):
        im1 = imlist[0]
        im1data = fits.getdata(im1)

        # gather info on the nods, make dictionaries to hold data
        nods = []
        nod_dict = {}
        nod_cubes = {}
        nod_meds = {}
        img_dict = {}

        for im in imlist:
            imgdata = fits.getdata(im)
            # this is to match a header to the final nod subtracted image
            # assumes that each image will sum to a unique value (ok assumption I think)
            img_dict[np.nansum(imgdata)] = im
            # this is to determine the nod from the header
            hdr = fits.getheader(im)
            nod_dict[im] = hdr['BEAM']
            if hdr['BEAM'] not in nods:
                nods.append(hdr['BEAM'])

        for nod in nods:
            ims = [key for (key, value) in nod_dict.items() if value == nod]
            nod_cube = np.zeros((len(ims), im1data.shape[0], im1data.shape[1]))
            for i in range(len(ims)):
                im = ims[i]
                imdata = fits.getdata(im)
                # add image to cube
                nod_cube[i] = imdata
            # add nod_cube to dictionary for later
            nod_cubes[nod] = nod_cube
            nod_meds[nod] = np.nanmedian(nod_cube, axis=0)

        for nod in nods:
            nod_cube = nod_cubes[nod]
            nod_coadd = np.nansum(nod_cube, axis=0)
            orig_str = img_dict[np.nansum(nod_cube[0])]
            new_str = orig_str.replace('.fit', '_COADD.fit')
            new_str = savepath+'\\'+new_str.split('\\')[-1]
            fits.writeto(new_str, nod_coadd, hdr, overwrite=True)

In [30]:
clioCoadd(reduced_datalists, datadir+'\\coadds')

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [02:26<00:00,  6.65s/it]
